In [ ]:
import datetime
import math
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from PyProbs import Probability as pr


In [ ]:
inicio = time()
N = 150
D = 50
n_clusters = 3  # Para seleccionar el número de clusters
# Condiciones iniciales:
I0 = 75  # número infectados inicial
R0 = 20  # número recuperados inicial
D0 = 10  # número muertos inicial
S0 = N-I0-D0-R0  # Número de susceptibles inicial

infection_radius = 15  # distancia para decidir si una persona se infecta o no
recovery_rate = 0.0001
death_rate = 0.03


In [ ]:
nn = N // n_clusters

R = D/2
# Generar aleatorios con distribución uniforme entre 0 y 1 (en número igual a n_clusters)
# para el ángulo de cada cluster usando np.random.uniform() y multiplicándolo por 2*math.pi
alpha = (np.random.uniform(0, 1, n_clusters)) * 2*math.pi

# Generar los radios r como R*np.sqrt(d), donde d son aleatorios con distribución uniforme (en número igual a n_clusters) entre 0 y 1
# Se usa la raíz cuadrada para generar una distribución circular uniforme, si no saldría una distribución con aros concéntricos.
r = R * np.sqrt(np.random.uniform(0, 1, n_clusters))

# Calcular las posiciones del centro de cada clusters convirtiendo de coordenadas polares a cartesianas
x0 = r*np.cos(alpha)
y0 = r*np.sin(alpha)
# Se crean arrays para las posiciones
xpos = np.zeros((n_clusters, nn))
ypos = np.zeros((n_clusters, nn))

# Por cada cluster (índice i),generar nn valores aleatorios xpos con distribución normal, usando np.random.normal(),
# que tenga media igual a x0[i] y desviación estándar igual a D/20. Generar nn valores aleatorios ypos con distribución normal, usando np.random.normal(),
# que tenga media igual a y0[i] y desviación estándar igual a D/20.
for i in range(n_clusters):
    xpos[i] = np.random.normal(loc=x0[i], scale=D/20, size=nn)
    ypos[i] = np.random.normal(loc=y0[i], scale=D/20, size=nn)
# En las líneas abajo se unen todos los xpos[i] y ypos[i] para crear un único array de xpos y ypos
xpos = np.reshape(xpos, (xpos.shape[0]*xpos.shape[1],))
ypos = np.reshape(ypos, (ypos.shape[0]*ypos.shape[1],))

# plt.scatter(xpos,ypos)

In [ ]:
# Definimos la condición (status) de las personas como "susceptible", "infected","recovered","dead"

# Definir un array donde aparezca un número de elementos "susceptible" igual a S0, un número de elementos "infected" igual a I0,
# un número de elementos "recovered" igual a R0 y un número de elementos "dead" igual a D0
# Sug.:usar np.repeat()

status = np.repeat(
    ["susceptible", "infected", "recovered", "dead"], [S0, I0, R0, D0])

# Crear un array de N elementos iguales a 0 (usar np.repeat())
iteration = np.repeat(0, N)
# Crear un array de N elementos que contenga valores de 1 a N
id = np.arange(1, N+1, 1)

In [ ]:
# Aquí se crea un dataframe con las columnas 'xpos', 'ypos', 'status','iteration' y 'id'
df = pd.DataFrame()
df['xpos'] = xpos
df['ypos'] = ypos
df['status'] = status
df['status'] = df['status'].astype('category')
df['iteration'] = iteration
df['id'] = id

In [ ]:
data = df
anim_df = data

In [ ]:
def euclidean_distance(a, b):
	return np.sqrt(sum((e1-e2)**2 for e1, e2 in zip(a,b)))

In [ ]:
infectados = np.array([I0])

In [ ]:
def infec(A, B):
    return (euclidean_distance(A, B) <= infection_radius)

In [ ]:
for k in range(6):
    de_xpos = np.random.normal(loc=0.0, scale=1, size=df[df['status']!="dead"].shape[0])
    de_ypos = np.random.normal(loc=0.0, scale=1, size=df[df['status']!="dead"].shape[0])
    data['xpos'] = df[df['status']!="dead"]['xpos'] + de_xpos
    data['ypos'] = df[df['status']!="dead"]['ypos'] + de_ypos
    data['iteration'] = df['iteration'] + 1
    for i in range(N):
        for j in range(N):
            if data['status'][i] == 'infected':
                if pr.Prob(recovery_rate):
                    data['status'][i] = 'recovered'
                if pr.Prob(death_rate):
                    data['status'][i] = 'dead'
                elif infec([data['xpos'][i], data['ypos'][i]], [data['xpos'][j], data['ypos'][j]]):
                    data['status'][j] = 'infected'
                    infectados+=np.array(I0)
    anim_df = pd.concat([anim_df, data])



In [ ]:
fig = px.scatter(anim_df, x="xpos", y="ypos", animation_frame="iteration", animation_group="id",
                 color="status")

fig.show()
print('Tiempo de ejecucion: ', datetime.timedelta(seconds=time()-inicio))

Tiempo de ejecucion:  0:00:13.049918


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=50d49ac8-895c-4e8f-b0c0-c54a516e012e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>